In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--conf spark.cassandra.connection.host=cassandra --packages com.datastax.spark:spark-cassandra-connector_2.11:2.0.2 pyspark-shell'

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [3]:
sc = SparkContext(appName="BigDataRiver")
sc.setLogLevel("WARN")
sc.setCheckpointDir('checkpoint/')
sql = SQLContext(sc)

In [4]:
def usersWhoBoughtXAlsoBought(df):
    productDf = df.select('user_id', 'product')
    otherProductDf = productDf.toDF('user_id', 'other_product')
    matchedProductsDf = productDf.join(otherProductDf, otherProductDf['user_id'] == productDf['user_id'], 'inner').\
            filter("`product` != `other_product`").select('product','other_product').\
            groupby('product','other_product').count().toDF("product","other_product","count")
    return matchedProductsDf

In [5]:
def selectTopProducts(df):
    df.registerTempTable("products")
    topProductsDf = sql.sql("""
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY product ORDER BY count DESC) rn
        FROM products
    """).where("rn <= 5").groupBy("product").agg(F.collect_list("other_product").alias("other_products"))
    return topProductsDf

In [6]:
def processBatch():
    allUserProductsDf = sql.read.format("org.apache.spark.sql.cassandra").\
        options(table="all_user_products", keyspace="bdr").load().cache()
    
    topDf = selectTopProducts(usersWhoBoughtXAlsoBought(allUserProductsDf))
    
    topDf.show()
            
    topDf.write.format("org.apache.spark.sql.cassandra").\
        mode('append').options(table="top_other_products_batch", keyspace="bdr").save()

In [7]:
processBatch()

+-------+--------------------+
|product|      other_products|
+-------+--------------------+
|     26|[84, 58, 73, 92, 96]|
|     29|[61, 62, 36, 30, 82]|
|     65|[89, 52, 96, 66, 87]|
|     19|[33, 41, 98, 28, 95]|
|     54|[10, 15, 79, 39, 18]|
|      0|[96, 83, 41, 95, 56]|
|     22|[64, 46, 82, 58, 38]|
|      7|[15, 85, 91, 90, 39]|
|     77|[91, 16, 51, 50, 21]|
|     34|[36, 82, 31, 58, 63]|
|     50| [47, 10, 91, 6, 32]|
|     94|[91, 84, 41, 15, 12]|
|     57|[12, 81, 61, 60, 30]|
|     32|[15, 10, 76, 50, 48]|
|     43|[89, 82, 85, 68, 30]|
|     84|[92, 26, 44, 36, 96]|
|     31|[89, 78, 34, 25, 80]|
|     39|[91, 15, 48, 59, 32]|
|     98|[68, 97, 37, 41, 21]|
|     25|[64, 11, 82, 97, 58]|
+-------+--------------------+
only showing top 20 rows

